## Libraries and Data

In [ ]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

!pip install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.5/158.5 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 27.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 17.5 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 4.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (73

In [ ]:
import pandas as pd
import spacy
from bertopic import BERTopic
import re
import nltk.corpus
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
data = pd.read_csv('/content/Dataset-SA.csv')
data['sentiment_code'] = pd.Categorical(data.Sentiment).codes
data['sentiment_code'] = data['sentiment_code'].astype('Int64')
data.dropna(inplace = True)

<ipython-input-3-5d6881ff21c1>:1: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/Dataset-SA.csv')


In [ ]:
print(data.product_name.nunique())
data.product_name.value_counts()

557


product_name
Lakm?? Eyeconic Kajal Twin Pack??????????(Deep Black, 0.7 g)                                                                                                                              5000
Mi 5A 80 cm (32 inch) HD Ready LED Smart Android TV with Dolby Audio (2022 Model)                                                                                                         2205
Mi 3i 10000 mAh Power Bank (Fast Charging, 18W)ÐÒ?ÐÓ®ÐÂ ÐÒ?ÐÓ®ÐÂ (Blue, Lithium Polymer)                                                                                          2006
Singer FM 1409 Electric Sewing MachineÐÒ?ÐÓ®ÐÂ ÐÒ?ÐÓ®ÐÂ ( Built-in Stitches 9)                                                                                                    2003
POCO C31 (Royal Blue, 64 GB)?ÐÒÐÒ?ÐÒÐÒ(4 GB RAM)                                                                                                                                  1999
                                

## Pre-process Text

In [ ]:
# define function to clean text

stop = stopwords.words('english')
add_stopwords = ['aqua', 'black', 'blue', 'fuchsia', 'gray', 'green', 'lime', 'maroon', 'navy', 'olive', 'purple', 'red', 'silver', 'teal', 'white', 'yellow', 'pack', 'light']
stop.extend(add_stopwords)

lemmatizer = WordNetLemmatizer()


def clean_text(text):

  text = text.lower()
  text = re.sub(r"(@\[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", text) # Punctuations, URLs and @
  text = " ".join([word for word in text.split() if word not in (stop)]) # Stopwords
  text = " ".join([lemmatizer.lemmatize(word) for word in text.split()]) # Stemming

  return(text)


In [ ]:
data_sub = data[['product_name', 'Summary']].copy(deep = True)
data_sub = data_sub.applymap(clean_text)
data_sub

,product_name,Summary
0,candes 12 l roompersonal air coolerwhite elega...,great cooler excellent air flow price amazing ...
1,candes 12 l roompersonal air coolerwhite elega...,best budget 2 fit cooler nice cooling
2,candes 12 l roompersonal air coolerwhite elega...,quality good power air decent
3,candes 12 l roompersonal air coolerwhite elega...,bad product fan
4,candes 12 l roompersonal air coolerwhite elega...,ok ok product
...,...,...
152289,home sizzler 214 cm 702 ft polyester semi tran...,low quality thing written room darkening trans...
152290,home sizzler 214 cm 702 ft polyester semi tran...,excellent
152291,home sizzler 214 cm 702 ft polyester semi tran...,beautiful n lovely
152292,home sizzler 214 cm 702 ft polyester semi tran...,good quality please add 8 foot length


## Topic Modelling

In [ ]:
# Topic modelling based on product name text

docs_product_name = data_sub.product_name
nlp = spacy.load('en_core_web_sm', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

topic_model_product_name = BERTopic(embedding_model=nlp, nr_topics=50)
topics, probs = topic_model_product_name.fit_transform(docs_product_name)

fig = topic_model_product_name.visualize_topics()
fig.show()


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid = os.fork()


In [ ]:
# Topic modelling based on Summary text

docs_Summary = data_sub.Summary
# nlp = spacy.load('en_core_web_sm', exclude=['tagger', 'parser', 'ner', 'attribute_ruler', 'lemmatizer'])

topic_model_Summary = BERTopic(embedding_model=nlp, nr_topics=20)
topics, probs = topic_model_Summary.fit_transform(docs_Summary)

fig = topic_model_Summary.visualize_topics()
fig.show()


/usr/local/lib/python3.10/dist-packages/joblib/externals/loky/backend/fork_exec.py:38: RuntimeWarning:

os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.

